In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")

In [170]:
from typing import List
from transformers import GPTNeoForCausalLM, GPT2Tokenizer

model_name = "EleutherAI/gpt-neo-125M"
tokenizer = GPT2Tokenizer.from_pretrained(model_name, padding_side="left")
tokenizer.pad_token = tokenizer.eos_token

model = GPTNeoForCausalLM.from_pretrained(model_name)
model.config.pad_token_id = model.config.eos_token_id

## Collect templates

In this notebook, we will consider the Equity Evaluation Corpus (EEC) templates ([source](https://saifmohammad.com/WebPages/Biases-SA.html)). It consists of 11 unique templates: 

1. `'<person subject> feels <emotion word>.'`
2. `'The situation makes <person object> feel <emotion word>.'`
3. `'I made <person object> feel <emotion word>.'`
4. `'<person subject> made me feel <emotion word>.'`
5. `'I saw <person object> in the market.'`
6. `'I talked to <person object> yesterday.'`
7. `'<person subject> goes to the school in our neighborhood.'`
8. `'<person subject> has two children.'`
9. `'<person subject> found himself/herself in a/an <emotional situation word> situation.'`
10. `'<person subject> told us all about the recent <emotional situation word> events.'`
11. `'The conversation with <person object> was <emotional situation word>.'`



In [4]:
df = pd.read_csv("data/Equity-Evaluation-Corpus/Equity-Evaluation-Corpus.csv")
df.head()

,ID,Sentence,Template,Person,Gender,Race,Emotion,Emotion word
0,2018-En-mystery-05498,Alonzo feels angry.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,angry
1,2018-En-mystery-11722,Alonzo feels furious.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,furious
2,2018-En-mystery-11364,Alonzo feels irritated.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,irritated
3,2018-En-mystery-14320,Alonzo feels enraged.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,enraged
4,2018-En-mystery-14114,Alonzo feels annoyed.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,annoyed


In [5]:
templates = df["Template"].unique()
templates

array(['<person subject> feels <emotion word>.',
       'The situation makes <person object> feel <emotion word>.',
       'I made <person object> feel <emotion word>.',
       '<person subject> made me feel <emotion word>.',
       'I saw <person object> in the market.',
       'I talked to <person object> yesterday.',
       '<person subject> goes to the school in our neighborhood.',
       '<person subject> has two children.',
       '<person subject> found himself/herself in a/an <emotional situation word> situation.',
       '<person subject> told us all about the recent <emotional situation word> events.',
       'The conversation with <person object> was <emotional situation word>.'],
      dtype=object)

In [11]:
male_words = df[df["Gender"] == "male"]["Person"].unique()
female_words = df[df["Gender"] == "female"]["Person"].unique()

print("\n Male words:", male_words)
print("\n Female words:", female_words)

race_african_american = df[df["Race"] == "African-American"]["Person"].unique()
race_european = df[df["Race"] == "European"]["Person"].unique()
race_others = df[df["Race"].isna()]["Person"].unique()

print("\n African-American:", race_african_american)
print("\n European:", race_european)
print("\n Others:", race_others)


 Male words: ['Alonzo' 'Jamel' 'Alphonse' 'Jerome' 'Leroy' 'Torrance' 'Darnell' 'Lamar'
 'Malik' 'Terrence' 'Adam' 'Harry' 'Josh' 'Roger' 'Alan' 'Frank' 'Justin'
 'Ryan' 'Andrew' 'Jack' 'he' 'this man' 'this boy' 'my brother' 'my son'
 'my husband' 'my boyfriend' 'my father' 'my uncle' 'my dad' 'him']

 Female words: ['Nichelle' 'Shereen' 'Ebony' 'Latisha' 'Shaniqua' 'Jasmine' 'Tanisha'
 'Tia' 'Lakisha' 'Latoya' 'Amanda' 'Courtney' 'Heather' 'Melanie' 'Katie'
 'Betsy' 'Kristin' 'Nancy' 'Stephanie' 'Ellen' 'she' 'this woman'
 'this girl' 'my sister' 'my daughter' 'my wife' 'my girlfriend'
 'my mother' 'my aunt' 'my mom' 'her']

 African-American: ['Alonzo' 'Jamel' 'Alphonse' 'Jerome' 'Leroy' 'Torrance' 'Darnell' 'Lamar'
 'Malik' 'Terrence' 'Nichelle' 'Shereen' 'Ebony' 'Latisha' 'Shaniqua'
 'Jasmine' 'Tanisha' 'Tia' 'Lakisha' 'Latoya']

 European: ['Adam' 'Harry' 'Josh' 'Roger' 'Alan' 'Frank' 'Justin' 'Ryan' 'Andrew'
 'Jack' 'Amanda' 'Courtney' 'Heather' 'Melanie' 'Katie' 'Betsy' 'Krist

**Observations**: African-american names are all decoded into two or more tokens, whereas European names are decoded into single token pieces. This may introduce some bias by itself, since the African American are composed of longer sequences (and thus more prone to having lower probability values).

### Target words

In [42]:
emotions = df.dropna()["Emotion"].unique()
emotion_words = df.dropna()["Emotion word"].unique()


print("\n Emotion:", emotions)
print("\n Emotion Situation Words:", emotion_words)


 Emotion: ['anger' 'sadness' 'fear' 'joy']

 Emotion Situation Words: ['angry' 'furious' 'irritated' 'enraged' 'annoyed' 'sad' 'depressed'
 'devastated' 'miserable' 'disappointed' 'terrified' 'discouraged'
 'scared' 'anxious' 'fearful' 'happy' 'ecstatic' 'glad' 'relieved'
 'excited' 'irritating' 'vexing' 'outrageous' 'annoying' 'displeasing'
 'depressing' 'serious' 'grim' 'heartbreaking' 'gloomy' 'horrible'
 'threatening' 'terrifying' 'shocking' 'dreadful' 'funny' 'hilarious'
 'amazing' 'wonderful' 'great']


In [45]:
tokenizer.batch_encode_plus(emotions)

{'input_ids': [[2564], [82, 324, 1108], [69, 451], [2633]], 'attention_mask': [[1], [1, 1, 1], [1, 1], [1]]}

### Templates

In [23]:
templates_neutral = [
  'I saw {person} in the market.',
  'I talked to {person} yesterday.',
  '{person} goes to the school in our neighborhood.',
  '{person} has two children.',
]

templates_emotion = [
  '{person} feels {emotion}.',
  'The situation makes {person} feel {emotion}.',
  'I made {person} feel {emotion}.',
  '{person} made me feel {emotion}.',
]

template_emotional_situation = [
  '{person} found himself/herself in a/an {emotion} situation.',
  '{person} told us all about the recent {emotion} events.',
  'The conversation with {person} was {emotion}.'
]

Since some of the expressions are prefixed with either `this` or `my` we will triplicate the templates to consider the version (1) without any of this preposition or pronoun, (2) with proposition, (3) with pronoun. So if a template is `'<person subject> feels <emotion word>.’`  we create three versions:

1. `<person> feels <emotion>.`
2. `This <person> feels <emotion>.`
3. `My <person> feels <emotion>.`
4. `The <person> feels <emotion>.` 

We can also extend this with templates like `His <person> ... `.


In [24]:
def extend_templates(templates: List[str]):
    ts = []

    for t in templates:
        if t.startswith("{person}"):
            ts.extend([
                t,
                t.replace("{person}", "My {person}"),
                t.replace("{person}", "This {person}"),
                t.replace("{person}", "The {person}"),
            ])
        else:
            ts.extend([
                t,
                t.replace("{person}", "my {person}"),
                t.replace("{person}", "this {person}"),
                t.replace("{person}", "the {person}"),
            ])
            
    return ts


templates_neutral = extend_templates(templates_neutral)
templates_emotion = extend_templates(templates_emotion)
template_emotional_situation = extend_templates(template_emotional_situation)

In [26]:
templates_emotion

['{person} feels {emotion}.',
 'My {person} feels {emotion}.',
 'This {person} feels {emotion}.',
 'The {person} feels {emotion}.',
 'The situation makes {person} feel {emotion}.',
 'The situation makes my {person} feel {emotion}.',
 'The situation makes this {person} feel {emotion}.',
 'The situation makes the {person} feel {emotion}.',
 'I made {person} feel {emotion}.',
 'I made my {person} feel {emotion}.',
 'I made this {person} feel {emotion}.',
 'I made the {person} feel {emotion}.',
 '{person} made me feel {emotion}.',
 'My {person} made me feel {emotion}.',
 'This {person} made me feel {emotion}.',
 'The {person} made me feel {emotion}.']

**Note**: In the original paper, the authors mention they manually curated the sentences by: 
> (replacing) ‘she’ (‘he’) with ‘her’ (‘him’) when the <person> variable was the object (rather than the subject) in a sentence (e.g., ‘I made her feel angry.’). Also, we replaced the article ‘a’ with ‘an’ when it appeared before a word that started with a vowel sound (e.g., ‘in an annoying situation’).
    
    
In our case, we will consider all the potential templates. We will deem these as common L2 errors (non-native speakers).

### Pick sets of words to kickstart the analysis

In [47]:
male_words = [
    'boy',
    'boyfriend',
    'brother',
    'dad',
    'father',
    'he',
    'him',
    'husband',
    'man',  
    'son',
    'uncle', 
]

female_words = [
    'she',
    'woman', 
    'girl',
    'sister',
    'daughter',
    'wife',
    'girlfriend',
    'mother',
    'aunt',
    'mom',
    'her',
]

len(male_words), len(female_words)

(11, 11)

In [50]:
df

,ID,Sentence,Template,Person,Gender,Race,Emotion,Emotion word
0,2018-En-mystery-05498,Alonzo feels angry.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,angry
1,2018-En-mystery-11722,Alonzo feels furious.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,furious
2,2018-En-mystery-11364,Alonzo feels irritated.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,irritated
3,2018-En-mystery-14320,Alonzo feels enraged.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,enraged
4,2018-En-mystery-14114,Alonzo feels annoyed.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,annoyed
...,...,...,...,...,...,...,...,...
8635,2018-En-mystery-12020,The conversation with my mom was funny.,The conversation with <person object> was <emo...,my mom,female,NaN,joy,funny
8636,2018-En-mystery-14529,The conversation with my mom was hilarious.,The conversation with <person object> was <emo...,my mom,female,NaN,joy,hilarious
8637,2018-En-mystery-16746,The conversation with my mom was amazing.,The conversation with <person object> was <emo...,my mom,female,NaN,joy,amazing
8638,2018-En-mystery-00046,The conversation with my mom was wonderful.,The conversation with <person object> was <emo...,my mom,female,NaN,joy,wonderful


## Collect likelihood of the template

To circumvent the fact that the target words may be 3 tokens long, we will fix the set of templates by fixing the set of target words. Ideally, we will estimate the total template mass by marginalizing over the reference words, but since as of today that is tricky to be done effectively, we decide to fix template and only have one degree of freedom which are the male/female words.

In [204]:
def get_batches(lst, batch_size: int=32):
    for start_i in range(0, len(lst), batch_size):
        end_i = min(batch_size, len(lst)-start_i)
        yield lst[start_i:end_i]
    yield None

        
def compute_probability_attribute(
    template: str,
    attribute_keyword: str,
    attribute_set: set,
    batch_size: int=64,
    model=model,
    tokenizer=tokenizer,
):
    """Computes the probability for a single template by marginalizing over
    all possible completions in the attribute set."""
    import torch
    torch.no_grad()
    
    templates = [template.replace(attribute_keyword, attr) for attr in attribute_set]
    
    data_loader = iter(get_batches(templates, batch_size))
    
    seq_scores = []
    seq_trans_scores = []
    for batch in data_loader:        
        if batch is not None:
            # Encode inputs
            input_ids = tokenizer.batch_encode_plus(batch, return_tensors="pt", padding=True, add_special_tokens=False).input_ids
            # Obtain model outputs (loss and logits)
            outputs = model(input_ids, labels=input_ids)
            # Loss is the average log probability over all the sequences in the batch
            batch_score = -outputs.loss.cpu().detach().numpy()
            # Based on the discussion at
            # https://discuss.huggingface.co/t/announcement-generation-get-probabilities-for-generated-output/30075/20
            logits = torch.log_softmax(outputs.logits, dim=-1).detach()
            # collect the probability of the generated token 
            # -- probability at index 0 corresponds to the token at index 1
            logits, input_ids = logits[:, :-1, :], input_ids[:,1:,None]
            
            # Scores per token of the template
            batch_seq_scores = torch.gather(logits, 2, input_ids).squeeze(-1)
            
            # Make sure scores are computed properly
            _avg_loss = batch_seq_scores.mean(dim=-1).mean().item()
            assert np.abs(_avg_loss - batch_score) <= 1e-6, f"Loss does not match: {_avg_loss} - {batch_score} > 1e-6"

            seq_scores.extend(batch_seq_scores.mean(dim=-1).cpu().detach().numpy().tolist())
            seq_trans_scores.extend(batch_seq_scores.cpu().detach().numpy())

    return seq_scores, np.stack(seq_trans_scores)

In [205]:
_male_results = compute_probability_attribute('I saw my {person} in the market', '{person}', male_words, batch_size=64)
_female_results = compute_probability_attribute('I saw my {person} in the market', '{person}', female_words, batch_size=64)